In [ ]:
!pip install ollama

Generating unit tests for code is a very tedious task and often takes a significant effort from the developers to write good test cases. There are various tools that are available for automated test generation, such as EvoSuite, which uses evolutionary algorithms to generate test cases. However, the test cases that are being generated are not natural and often developers do not prefer to add them to their test suite. Whereas Large Language Models (LLM) being trained with developer-written code it has a better affinity towards generating more natural code--more readable, maintainable code. In this excercise, we will show we can leverage LLMs to generate test cases with the help of CLDK. 

For simplicity, we will cover certain aspects of test generation and provide some context information to LLM for better quality of test cases. In this exercise, we will generate a unit test for a non-private method from a Java class and provide the focal method body and the signature of all the constructors of the class so that LLM can understand how to create an object of the focal class during the setup phase of the tests. Also, we will ask LLMs to generate ```N``` number of test cases, where ```N``` is the cyclomatic complexity of the focal method. The intuition is that one test may not be sufficient for covering fairly complex methods, and a cyclomatic complexity score can provide some guidance towards that. 

(Step 1) First, we will import all the necessary libraries

In [ ]:
import ollama
from cldk import CLDK
from cldk.analysis import AnalysisLevel

(Step 2) Second, we will form the prompt for the model, which will include all the constructor signarures, and the body of the focal method.

In [ ]:
def format_inst(focal_method_body, focal_method, focal_class, constructor_signatures, cyclomatic_complexity, language):
    """
    Format the instruction for the given focal method and class.
    """
    inst = f"Question: Can you generate {cyclomatic_complexity} unit tests for the method `{focal_method}` in the class `{focal_class}` below?\n"

    inst += "\n"
    inst += f"```{language}\n"
    inst += "```\n"
    inst += "public class {focal_class} {"
    inst += f"<|constructors|>\n{constructor_signatures}\n<|constructors|>\n"
    inst += f"<|focal method|>\n {focal_method_body} \n <|focal method|>\n" 
    inst += "}"
    inst += "```\n"
    inst += "Answer:\n"
    return inst

(Step 3) Third, use ollama to call LLM (in case Granite 8b).

In [ ]:
def prompt_ollama(message: str, model_id: str = "granite-code:20b-instruct") -> str:
    """Prompt local model on Ollama"""
    response_object = ollama.generate(model=model_id, prompt=message)
    return response_object["response"]

(Step 4) Fourth, collect all the information needed for each method. In this process, we go through all the classes in the application, and then for each class, we collect the signature of all the constructors. If there is no constructor present, we add the signature of the default constructor. Then, we go through all the non-private methods of the class and formulate the prompt using the constructor and the method information. Finally, we use the prompt to call LLM and get the final output.

In [ ]:
# Create a new instance of the CLDK class
cldk = CLDK(language="java")
# Create an analysis object over the java application. Provide the application path using JAVA_APP_PATH
analysis = cldk.analysis(project_path="JAVA_APP_PATH", analysis_level=AnalysisLevel.symbol_table)
# Go through all the classes in the application
for class_name in analysis.get_classes():
    class_details  = analysis.get_class(qualified_class_name=class_name)
    # Generate test cases for non-interface and non-abstract classes
    if not class_details.is_interface and 'abstract' not in class_details.modifiers:
        # Get all constructor signatures
        constructor_signatures = ''
        for method in analysis.get_methods_in_class(qualified_class_name=class_name):
            method_details = analysis.get_method(qualified_class_name=class_name, qualified_method_name=method)
            if method_details.is_constructor:
                constructor_signatures += method_details.signature + '\n'
        # If no constructor present, then add the signature of the default constructor
        if constructor_signatures=='':
            constructor_signatures = f'public {class_name} ()'
        # Go through all the methods in the class
        for method in analysis.get_methods_in_class(qualified_class_name=class_name):
            # Get the method details
            method_details = analysis.get_method(qualified_class_name=class_name, qualified_method_name=method)
            # Generate test cases for non-private methods
            if 'private' not in method_details.modifiers and not method_details.is_constructor:
                # Gather all the information needed for the prompt, which are focal method body, focal method name, focal class name, constructor signature, and cyclomatic complexity
                prompt = format_inst(focal_method_body=method_details.code,
                                     focal_method=method,
                                     focal_class=class_name,
                                     constructor_signatures=constructor_signatures,
                                     cyclomatic_complexity=method_details.cyclomatic_complexity)
                # Prompt the local model on Ollama
                llm_output = prompt_ollama(
                    message=prompt,
                    model_id="granite-code:20b-instruct",
                )
        
                # Print the instruction and LLM output
                print(f"Instruction:\n{prompt}")
                print(f"LLM Output:\n{llm_output}")